Hacky notebook for tesing different models

Not used in final experiments

In [1]:
cd ../deepsalience/

/home/rmb456/repos/multif0/deepsalience


In [2]:
from __future__ import print_function

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

import glob
import librosa
import mir_eval
import numpy as np
import os
import pandas

import compute_training_data as C

def test_path():
    return '/scratch/rmb456/multif0_ismir2017/test_data/'


def get_single_test_prediction(npy_file, model):
    input_hcqt = np.load(npy_file).transpose(1, 2, 0)[np.newaxis, :, :, :]

    n_t = input_hcqt.shape[2]
    t_slices = list(np.arange(0, n_t, 5000))
    output_list = []
    for t in t_slices:
        output_list.append(
            model.predict(input_hcqt[:, :, t:t+5000, :])[0, :, :]
        )

    predicted_output = np.hstack(output_list)
    return predicted_output, input_hcqt


def pitch_activations_to_mf0(pitch_activation_mat, thresh):
    freqs = C.get_freq_grid()
    times = C.get_time_grid(pitch_activation_mat.shape[1])

    idx = np.where(pitch_activation_mat >= thresh)

    freqs = [[] for _ in range(len(times))]
    for f, t in zip(idx[0], idx[1]):
        freqs[t].append(freqs[f])

    freqs = [np.array(lst) for lst in freqs]
    return times, freqs


def save_multif0_output(times, freqs, output_path):
    with open(output_path, 'w') as fhandle:
        csv_writer = csv.writer(fhandle, delimiter='\t')
        for t, f in zip(times, freqs):
            csv_writer.writerow([t] + f)


def score_on_test_set(test_set_name, model, save_path):

    # get files for this test set
    test_set_path = os.path.join(test_path(), test_set_name)
    test_npy_files = glob.glob(os.path.join(test_set_path, '*.npy'))

    all_scores = []
    for npy_file in test_npy_files:
        # get input npy file and ground truth label pair
        file_keys = os.path.basename(npy_file).replace('-', '_').split('_')[:2]
        label_file = glob.glob(
            os.path.join(test_set_path, "{}*{}*.txt".format(file_keys[0], file_keys[1]))
        )[0]
        
        # generate prediction on numpy file
        predicted_output, _ = get_single_test_prediction(npy_file, model)
        
        # save prediction
        np.save(
            os.path.join(
                save_path,
                "{}_{}_prediction.npy".format(file_keys[0], file_keys[1])
            ),
            predicted_output.astype(np.float32)    
        )
        
        # get multif0 output from prediction
        est_times, est_freqs = pitch_activations_to_mf0(predicted_output, 0.5)
        
        # save multif0 output
        save_multif0_output(
            est_times, est_freqs,
            os.path.join(
                save_path,
                "{}_{}_prediction.txt".format(file_keys[0], file_keys[1])
            )
        )
        
        # load ground truth labels
        ref_times, ref_freqs = mir_eval.io.load_ragged_time_series(label_file)
        
        # get multif0 metrics and append
        scores = mir_eval.multipitch.evaluate(ref_times, ref_freqs, est_times, est_freqs)
        scores['track'] = pair_key
        all_scores.append(scores)
    
    # save scores to data frame
    scores_path = os.path.join(save_path, '{}_all_scores.csv'.format(test_set_name))
    score_summary_path = os.path.join(
        save_path, "{}_score_summary.csv".format(test_set_name)
    )
    df = pandas.DataFrame(all_scores)
    df.to_csv(scores_path)
    df.describe().to_csv(score_summary_path)
    print(df.describe())

In [3]:
import experiment

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7612483073847078652
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 11332668621
locality {
  bus_id: 1
}
incarnation: 12398025123977076967
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:05:00.0"
]


Using TensorFlow backend.


In [4]:
from __future__ import print_function
import keras
from keras.models import Model
from keras.layers import Dense, Input, Reshape, Lambda
from keras.layers.convolutional import Conv2D
from keras.layers.normalization import BatchNormalization
from keras import backend as K
import os

In [14]:
input_shape = (None, None, 6)
inputs = Input(shape=input_shape)

y1 = Conv2D(64, (5, 5), padding='same', activation='relu', name='bendy1')(inputs)
y1a = BatchNormalization()(y1)
y2 = Conv2D(128, (3, 3), padding='same', activation='relu', name='harmonics')(y1a)
y2a = BatchNormalization()(y2)
y3 = Conv2D(256, (3, 3), padding='same', activation='relu', name='smoothy1')(y2a)
y3a = BatchNormalization()(y3)
y4 = Conv2D(1, (1, 1), padding='same', activation='sigmoid', name='squishy')(y3a)
predictions = Lambda(lambda x: K.squeeze(x, axis=3))(y4)

model = Model(inputs=inputs, outputs=predictions)

In [15]:
experiment.train(model, '../experiment_output/notebook_test6')

________________________________________________________________________________
Layer (type)                        Output Shape                    Param #     
input_8 (InputLayer)                (None, None, None, 6)           0           
________________________________________________________________________________
bendy1 (Conv2D)                     (None, None, None, 64)          9664        
________________________________________________________________________________
harmonics (Conv2D)                  (None, None, None, 128)         73856       
________________________________________________________________________________
smoothy1 (Conv2D)                   (None, None, None, 256)         295168      
________________________________________________________________________________
squishy (Conv2D)                    (None, None, None, 1)           257         
________________________________________________________________________________
lambda_4 (Lambda)           

KeyboardInterrupt: 

In [ ]:
model.load_weights("../experiment_output/multif0_exper7_3/multif0_exper7_3.pkl")

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
# plot filters
conv_layer = model.get_layer(name='distribution')
weights = conv_layer.get_weights()
weight_array = weights[0]
plt.figure(figsize=(20, 10))
for i in range(8):
    plt.subplot(1, 8, i+1)
#     plt.plot(weight_array[:, :, 0, i])
    plt.imshow(weight_array[:, :, 0, i], origin='lower', vmin=weight_array.min(), vmax=weight_array.max())
    plt.axis('auto')

plt.show()

In [ ]:
# plot filters
conv_layer = model.get_layer(name='bendy1')
weights = conv_layer.get_weights()
weight_array = weights[0]
plt.figure(figsize=(15, 15))
for i in range(64):
    plt.subplot(8, 8, i+1)
    plt.imshow(weight_array[:, :, 1, i], origin='lower', vmin=weight_array.min(), vmax=weight_array.max())
    plt.axis('square')

plt.show()


In [ ]:
# plot filters
conv_layer = model.get_layer(name='deharm')
weights = conv_layer.get_weights()
weight_array = weights[0]
plt.figure(figsize=(15, 10))
for i in range(16):
    plt.subplot(2, 8, i+1)
    plt.imshow(weight_array[:, :, 0, i], origin='lower', vmin=weight_array.min(), vmax=weight_array.max())
    plt.axis('auto')


plt.show()